In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

print("Training data size", len(train_dataloader)*batch_size)
print("Test data size", len(test_dataloader)*batch_size)

for X,y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [01:01<00:00, 160200.18it/s]


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 52954.92it/s]


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:21<00:00, 75265.49it/s] 


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 553215.49it/s]

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw

Training data size 60032
Test data size 60032
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# Define the device

In [3]:
device = torch.device(
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print("We're using", device)

We're using cpu


# Define Model

In [4]:
class NN(nn.Module):    # Neurual Network
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes)
        )
    
    def forward(self, image_tensor):
        image_tensor = self.flatten(image_tensor)
        logits = self.linear_relu_stack(image_tensor)
        return logits


input_size = 28*28
hidden_size = 512
num_classes = 10

model = NN(input_size, hidden_size, num_classes).to(device)
print(model)

NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
len(train_dataloader)

938

# define learning rate, loss function and optimizer

In [8]:
learning_rate = 1e-3
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# Train Loop

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch_num, (X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_num > 0 and batch_num % 100 == 0:
            loss, current = loss.item(), batch_num * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# Test loop

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0

    for X,y in dataloader:
        X,y = X.to(device), y.to(device)
        pred= model(X)
        test_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# Train the model

In [12]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------")
    train(train_dataloader,model, loss_fn, optimizer)
    test(test_dataloader,model, loss_fn)
print("Done bangg")

Epoch 1
----------------------------
loss: 0.290817  [ 6400/60000]
loss: 0.203883  [12800/60000]
loss: 0.255108  [19200/60000]
loss: 0.155604  [25600/60000]
loss: 0.346649  [32000/60000]
loss: 0.124778  [38400/60000]
loss: 0.283174  [44800/60000]
loss: 0.299666  [51200/60000]
loss: 0.162034  [57600/60000]
Test error: 
 Accuracy: 95.7%, Avg loss: 0.136747 

Epoch 2
----------------------------
loss: 0.105226  [ 6400/60000]
loss: 0.068073  [12800/60000]
loss: 0.082446  [19200/60000]
loss: 0.055886  [25600/60000]
loss: 0.124918  [32000/60000]
loss: 0.042677  [38400/60000]
loss: 0.139233  [44800/60000]
loss: 0.101220  [51200/60000]
loss: 0.085084  [57600/60000]
Test error: 
 Accuracy: 97.6%, Avg loss: 0.075055 

Epoch 3
----------------------------
loss: 0.062121  [ 6400/60000]
loss: 0.062274  [12800/60000]
loss: 0.067503  [19200/60000]
loss: 0.032629  [25600/60000]
loss: 0.088793  [32000/60000]
loss: 0.029406  [38400/60000]
loss: 0.065263  [44800/60000]
loss: 0.090166  [51200/60000]
loss:

In [14]:
torch.save(model.state_dict(), "MNIST_NN_3_layers.pth")
print("saving model . . .")
model = NN(input_size, batch_size, num_classes).to(device)
model.load_state_dict(torch.load("MNIST_NN_3_layers.pth"))

model.eval()
for i in range(10):
    x,y = test_data[i][0], test_data[i][1]

    x = x.to(device)
    pred = model(x)
    predicted, actual = pred[0].argmax(0).item(), y
    print(f' Predicted "{predicted}" , Actual: "{actual}"')



saving model . . .
 Predicted "8" , Actual: "5"
 Predicted "8" , Actual: "0"
 Predicted "8" , Actual: "4"
 Predicted "8" , Actual: "1"
 Predicted "8" , Actual: "9"
 Predicted "8" , Actual: "2"
 Predicted "8" , Actual: "1"
 Predicted "8" , Actual: "3"
 Predicted "8" , Actual: "1"
 Predicted "8" , Actual: "4"


C:\Windows\Temp\ipykernel_15460\1993320087.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("MNIST_NN_3_layers.pth"))
